# Spaceship spawner

Spawns a spaceship in Space Engineers.

## Imports and setup

In [ ]:
from pcgsepy.common.api_call import get_base_values, GameMode, toggle_gamemode
from pcgsepy.common.vecs import Vec, Orientation
from pcgsepy.lsystem.structure_maker import LLStructureMaker
from pcgsepy.setup_utils import setup_matplotlib, get_default_lsystem
from pcgsepy.structure import Structure, place_structure, place_blocks
from pcgsepy.hullbuilder import HullBuilder

In [ ]:
setup_matplotlib(larger_fonts=False)

used_ll_blocks = [
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerInv',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
    'MyObjectBuilder_Gyro_LargeBlockGyro',
    'MyObjectBuilder_Reactor_LargeBlockSmallGenerator',
    'MyObjectBuilder_CargoContainer_LargeBlockSmallContainer',
    'MyObjectBuilder_Cockpit_OpenCockpitLarge',
    'MyObjectBuilder_Thrust_LargeBlockSmallThrust',
    'MyObjectBuilder_InteriorLight_SmallLight',
    'MyObjectBuilder_CubeBlock_Window1x1Slope',
    'MyObjectBuilder_CubeBlock_Window1x1Flat',
    'MyObjectBuilder_InteriorLight_LargeBlockLight_1corner'
]

lsystem = get_default_lsystem(used_ll_blocks=used_ll_blocks)

## Spaceship string

Define here the high-level spaceship string.

In [ ]:
spaceship_string = 'cockpitcorridorsimple(1)corridorsimple(1)[RotYcwZcorridorsimple(2)]corridorgyros(1)[RotYcwXcorridorsimple(2)]corridorsimple(1)corridorreactors(2)corridorsimple(1)[RotYccwXcorridorsimple(1)]corridorcargo(1)thrusters'

## Spaceship creation

In [ ]:
ml_string = lsystem.hl_solver.translator.transform(string=spaceship_string)
ll_solution = lsystem.ll_solver.solve(string=ml_string,
                                   iterations=1,
                                   strings_per_iteration=1,
                                   check_sat=False)[0]
base_position, orientation_forward, orientation_up = Vec.v3i(
            0, 0, 0), Orientation.FORWARD.value, Orientation.UP.value
structure = Structure(origin=base_position,
                      orientation_forward=orientation_forward,
                      orientation_up=orientation_up)
structure = LLStructureMaker(
    atoms_alphabet=lsystem.ll_solver.atoms_alphabet,
    position=base_position).fill_structure(structure=structure,
                                           string=ll_solution.string)
structure.sanify()

hull_builder = HullBuilder(erosion_type='bin',
                           apply_erosion=True,
                           apply_smoothing=False)
hull_builder.add_external_hull(structure=structure)

structure.show('')

## In-game placement

In [ ]:
base_position, orientation_forward, orientation_up = get_base_values()
# place_structure(structure=structure,
#                 position=base_position,
#                 orientation_forward=orientation_forward,
#                 orientation_up=orientation_up,
#                 batchify=False)
structure.update(
    origin=base_position,
    orientation_forward=orientation_forward,
    orientation_up=orientation_up,
)
toggle_gamemode(GameMode.PLACING)
all_blocks = structure.get_all_blocks(to_place=True)
for block in all_blocks:
    block.position = block.position.sum(base_position)
place_blocks(all_blocks, sequential=False)
toggle_gamemode(GameMode.EVALUATING)